In [7]:
import requests, os
from bs4 import BeautifulSoup
from src.downloader import download

def get_html(url):
    """Get raw HTML from a URL."""
    headers = {
        'Access-Control-Allow-Origin': '*',
        'Access-Control-Allow-Methods': 'GET',
        'Access-Control-Allow-Headers': 'Content-Type',
        'Access-Control-Max-Age': '3600',
        'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }
    req = requests.get(url, headers=headers)
    return req.text

def get_image(html):
    """Scrape share image."""
    image = []
    if html.find("meta", property="image"):
        image = html.find("meta", property="image").get('content')
    elif html.find("meta", property="og:image"):
        image = html.find("meta", property="og:image").get('content')
    elif html.find("meta", property="twitter:image"):
        image = html.find("meta", property="twitter:image").get('content')
    elif html.find("img", src=True):
        for img in html.find_all("img"):
            image.append(img.get('src'))
    return image


def check_word(word):
    def check(url):
        return (word in url)
    return check

In [14]:
keyword = "gadget electronique"
page = "2"
url = "https://www.amazon.fr/s?k={}&page={}&__mk_fr_FR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&ref=sr_pg_{}".format(keyword, page, page)

soup = BeautifulSoup(get_html(url))
# print(soup.prettify())
for image_url in list(filter(check_word("_AC_UL320_"), get_image(soup))):
    download(image_url, "imgs")

SSLError: HTTPSConnectionPool(host='m.media-amazon.com', port=443): Max retries exceeded with url: /images/I/61YC5v2tk7L._AC_UL320_.jpg (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1076)')))

In [1]:
import os, requests, time, shutil, glob
import bz2
import numpy as np
from multiprocessing.dummy import Pool
from src.azurekey import *

def search(search_term, azureKey, offset=0):
    # print('searching using bing: "'+search_term+'"')
    search_url = "https://api.bing.microsoft.com/v7.0/search"
    subscription_key = azureKey
    assert subscription_key
    headers = {"Ocp-Apim-Subscription-Key" : subscription_key}
    params  = {"q": search_term, "responseFilter": "Webpages","count":150, "offset":offset, "safeSearch":"Strict"}
    response = requests.get(search_url, headers=headers, params=params)
    response.raise_for_status()
    search_results = response.json()
    return search_results


def download(links_in, dir_name, n=None, log=set()):
    links = links_in[:n]
    # print('search results',len(links)
    if not os.path.exists(dir_name):
      os.mkdir(dir_name)

    def fetch(url, log=log):
      r=requests.get(url[0], stream=True)
      r.raw.decode_content = True # handle spurious Content-Encoding
      img = Image.open(r.raw)
      bits = str(imagehash.average_hash(img))
      if bits not in log:
        log |= set([bits])
        if len(log) % 100 == 0:
          print("{} images saved.".format(len(log)))
        img_file = dir_name+'/commercial_product_'+bits+".jpg"
        img.save(img_file)
        

    modLinks=[[links[i],i+1] for i in range(len(links))]
    Pool(10).map(fetch, modLinks)

    return log

In [2]:
search_results = search("electronics", azureKey, offset=0)

In [3]:
from IPython.display import HTML

rows = "\n".join(["""<tr>
                       <td><a href=\"{0}\">{1}</a></td>
                       <td>{2}</td>
                     </tr>""".format(v["url"], v["name"], v["snippet"])
                  for v in search_results["webPages"]["value"]])
HTML("<table>{0}</table>".format(rows))

Aven Electronics | Bienvenue dans le meilleur de l ...,Le meilleur de l'électroménager c'est chez Aven Electronics. Découvrez nos produits à travers différentes catégories et profitez d'une assistance totale.
Ei Electronics – FR,"Ei Electronics est une société entièrement irlandaise. Elle est le fruit du rachat, en mars 1988 par ses dirigeants, de General Electric dont le début d’activité remonte à 1963. La compagnie a connu une croissance constante en développant toute une gamme d’équipements innovants élaborés selon des procédés technologiques de pointe. Ces produits sont conçus pour répondre aux ..."
"Téléphonie, TV, Informatique, Électroménager, Audio, Video ...","LG Electronics vous propose son expertise en téléphonie mobile, produits audio vidéo, téléviseurs OLED et Super UHD 4K, Home cinéma, moniteurs et électroménager"
Electronics - Wikipedia,"Electronics comprises the physics, engineering, technology and applications that deal with the emission, flow and control of electrons in vacuum and matter. It uses active devices to control electron flow by amplification and rectification, which distinguishes it from classical electrical engineering which uses passive effects such as resistance, capacitance and inductance to control current flow."
Page d'accueil Electronic Arts - Site officiel EA,"Notre mission : inspirer le monde par nos jeux. Electronic Arts est un éditeur majeur de jeux sur consoles, PC et mobiles."
Amazon.com: Electronics,All Electronics Deals Best Sellers TV & Video Audio & Home Theater Computers Camera & Photo Wearable Technology Car Electronics & GPS Portable Audio Cell Phones Office Electronics Musical Instruments New Arrivals Trade-In
"Clubelec.com - composants électroniques, rachat surstock ...",CLUB ELECTRONICS - ISO 9001:2015 - est une société française de distribution de composants électroniques auprès des fabricants et des utilisateurs de systèmes électroniques. Nous intervenons sur le second marché des composants : quand vos fournisseurs habituels sont dans l'impossibilité de vous aider et de répondre à vos besoins d'approvisionnement. Nos fournisseurs et nos clients ...
Beats by Dre (France),"Achetez des casques, écouteurs, enceintes et accessoires sur BeatsbyDre.com. Son puissant et technologie audio de Beats by Dre. Livraison gratuite en 2 jours, tous les jours."
Omron France,"Omron France est un fournisseur de technologie dans les domaines de l'automatisation industrielle, de la santé et des composants électroniques."
"melectronics: électronique, multimédia, électroménager ...","Dans notre boutique en ligne, commandez des articles de marque et faites-les vous livrer directement à domicile ou gratuitement dans un magasin Migros"
Solutions innovantes pour le test des réseaux électriques ...,"OMICRON fournit à l'industrie électrique des produits et des services innovants pour le test, le diagnostic et la surveillance d'équipements."
